In [5]:
from modules.base_module import BaseModule

from modules.scouter import connectES as es
from modules.scouter import clusteringDoc as cl
from modules.scouter import N_gram as ng

import json

In [6]:
class TopicGenerator(BaseModule):
    
    def __init__(self, topic, out_path):
        super(TopicGenerator, self).__init__(topic, out_path)
        
    def process_data(self, date):
        
        indexed_cluster = es.keywordGetter(date)
        if indexed_cluster != False:
            return indexed_cluster
        else:
            qbody = es.ScouterHandler.make_date_query_body(date,("news_id","extContent","analyzed_text","analyzed_text.sentence.WSD.text",
                                                                 "analyzed_text.sentence.WSD.type","category","analyzed_title.sentence.WSD.text",
                                                                 "analyzed_title.sentence.WSD.type"))
            getDoc = es.ScouterHandler()
            docs  = getDoc.search(qbody,'newspaper')
            result = {'date': date,
               'doc count': len(docs)}
            clus = cl.cluteringDoc(5,0.95,docs)

           #centroivalue 용
           #ctime, c_result, c_cent_result = clus.runClus()
           #for i in c_cent_result:
           #    c_doc.append([i])

           #whole value 용
            ctime, c_result = clus.runClus()
            c_doc=list()
            for i in range(len(c_result)):
                analyzed_doc = list()
                for j in c_result[i]:
                    analyzed_doc.append(j[2])
                c_doc.append(analyzed_doc)

            '''analyzed_doc = list()
               for j in c_cent_result:
                   analyzed_doc.append(j)
               c_doc.append(analyzed_doc)'''

           #print(1)
            for i in range(5):
                cluster_uni, cluster_bi, cluster_tri=ng.ngram(c_doc[i])
                cluster_uni = cluster_uni.most_common(10)
                cluster_bi = cluster_bi.most_common(10)
                cluster_tri = cluster_tri.most_common(10)
               # a=sorted(cluster_uni.items(), key=lambda t:t[1], reverse=True)
               # b=sorted(cluster_bi.items(), key=lambda t:t[1], reverse=True)
               # c=sorted(cluster_tri.items(), key=lambda t:t[1], reverse=True)
                result['cluster'+str(i+1)+'unigram'] = cluster_uni
                result['cluster'+str(i+1)+'bigram'] = cluster_bi
                result['cluster'+str(i+1)+'trigram'] = cluster_tri
                
            return result

    
    

In [7]:
t = TopicGenerator('', '')
print(t.process_data('2019-05-03'))

Query: 2019-05-03
Scroll idx : 1 (1000 docs)
Scroll idx : 2 (252 docs)
Total retrieved Doc # :  1252

elapsed time taken for clustering:  124.88582015037537 seconds.
{'date': '2019-05-03', 'doc count': 1252, 'cluster1unigram': [('투자', 1231), ('금융', 918), ('시장', 713), ('기업', 613), ('분기', 552), ('거래', 549), ('증가', 545), ('주가', 515), ('상승', 464), ('대비', 460)], 'cluster1bigram': [('영업 이익', 237), ('국민 연금', 180), ('반 도체', 134), ('투자 의견', 132), ('동기 대비', 116), ('전년 동기', 112), ('금융 투자', 112), ('어린 이', 111), ('목표 주가', 105), ('대비 증가', 99)], 'cluster1trigram': [('전년 동기 대비', 104), ('매출 영업 이익', 60), ('이 투자 의견', 52), ('목표 주가 제시', 50), ('동기 대비 증가', 49), ('스마트 폰 사용', 42), ('신종 자본 증권', 41), ('최근 동안 매물', 41), ('동안 매물 분포', 41), ('대형 매물 눈', 41)], 'cluster2unigram': [('의견', 1827), ('목표', 1378), ('투자', 1241), ('제시', 1057), ('주가', 916), ('유지', 908), ('최근', 806), ('증권', 694), ('이번', 622), ('전체', 573)], 'cluster2bigram': [('목표 주가', 758), ('투자 의견', 747), ('의견 유지', 375), ('금융 투자', 290), ('제시 목표', 266), ('이번 제시',